In [1]:
import os
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import MemorySaver  # MemorySaver() suporta async
from langchain.messages import HumanMessage
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

e:\Projects\PersonalSite\PSWindows\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [3]:
# Model Config - habilite streaming
llm = ChatOpenAI(
    model="gpt-5-nano-2025-08-07",  # gpt-5-nano pode não existir, use gpt-4o-mini
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.7,
    streaming=True  # Necessário para token streaming
)

In [4]:
# MCP Config
mcp_config = {
    "github": {
        "transport": "streamable_http",  # transporte HTTP suportando headers
        "url": "https://api.githubcopilot.com/mcp/",  # você usaria o endpoint público de GitHub
        "headers": {
            "Authorization": f"Bearer {os.getenv('GITHUB_ACESS_TOKEN')}"
        }
    }   
}

In [5]:
# MCP Client
client = MultiServerMCPClient(mcp_config)
tools = await client.get_tools()  # Async tools

In [6]:
# ✅ MemorySaver() (não InMemorySaver) para async support
Agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=MemorySaver(),  # Suporta async tools corretamente
)

In [ ]:
thread = input("Nomeie a conversa: ")

In [10]:
config = {
    "configurable": {"thread_id": thread},
    "stream_mode": "values"  # Ou "updates", ["messages"], etc.
}

In [14]:
async for chunk in Agent.stream(
    {"messages": [HumanMessage(content="Busca pra mim o repositório mais recente do Baldros.")]},  # ✅ dict com 'messages'
    config=config  # ✅ stream_mode dentro de config
):
    # ✅ Para "values"/"updates": chunk é dict[node_name, state_delta]
    # Acesse mensagens: chunk['model']['messages'][-1].content ou similar
    if 'model' in chunk:
        last_msg = chunk['model']['messages'][-1]
        print(last_msg.content if hasattr(last_msg, 'content') else str(last_msg))
    else:
        print(chunk)

TypeError: 'async for' requires an object with __aiter__ method, got generator